# Задание

В этом задании вам предстоит использовать данные о выживаемости на Титанике для построения модели бинарной классификации для признака Survived.

Считайте датасет из файла train.csv (это данные о выживаемости на Титанике)

Выберите и обоснуйте метрику для измерения качества (accuracy/precision/recall/f1-score/fbeta-score/roc-auc и т.д.). В рамках данного пункта необходимо подобрать наиболее релевантную метрику или набор метрик для вашей задачи, написав краткое обоснование (1-2 предложения) - 2 балла

Постройте бейзлайн и ML-модель классификации (LogisticRegression или любая другая, которая вам кажется подходящей) и оцените их качество с помощью выбранной метрики


Произведено разбиение датасета на тренировочную/тестовую выборки - 1 балл

Произведено измерение качества константного предсказания (например, наиболее частотный класс/случайное предсказание) - 1 балл

ML-модель обучена на тренировочной выборке, учтены особенности предобработки данных для модели, если они есть - 3 балла

Произведено измерение качества на отложенной выборке с использованием ранее выбранной метрики - 1 балл

Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - 2 балла


# Анализ датасета и выбор метрики

In [162]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [163]:
filename = "/content/gdrive/MyDrive/python для анализа данных/train.csv"

In [164]:
df = pd.read_csv(filename)

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [166]:
df["Ticket"].value_counts()

,count
Ticket,
347082,7
1601,7
CA. 2343,7
3101295,6
CA 2144,6
...,...
PC 17590,1
17463,1
330877,1


Не будем учитывать PassengerId, ticket, name, cabin - первые три слишком уникальные, последнее содержит ту мач пропусков. Дропаем остальные пропуски




In [167]:
df1 = df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"]).dropna().reset_index(drop=True)
df1.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [168]:
df1["Survived"].value_counts()

,count
Survived,
0,424
1,288


Есть некоторый дисбаланс классов, поэтому **accuracy** в качестве метрики - **не берем**.

**Recall, Precision** имеет смысл оценивать в совокупности, но удобнее, когда качество выражается одним числом. Поскольку нет дальнейших практических указаний, для чего будет использоваться модель, то мы не можем сказать, какой из этих параметров важнее. Будем считать, что они одинаково важны, так что нам **подойдет F1 мера**.

Но F1-score концентрируется на верном определении класса 1. Поэтому в дополнение к этой метрике хочется взять метрику, которая бы балансировала оба класса. Для этого **возьмем ROC-AUC**. Хоть он тоже чувствителен к дисбалансу классов, но в нашем случае дисбаланс не сильный, поэтому можем попробовать.

NB: в реальной задаче мы скорее всего понимали бы, важен нам только класс 1 или оба, и исходя из этого выбрали бы соответствующую метрику. Но при обучении интересно сравнить разные варианты.

ИТОГ: берем F1-score, ROC-AUC

# Подготовка данных, разбиение датасета

Далее надо подготовить данные: заменить строковые значения на категориальные.  В случае пола достаточно применить label encoding, для embarked - one-hot encoding и дропнуть один из столбцов, чтобы избежать линейной зависимости.

In [169]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [170]:
le = LabelEncoder()
res = le.fit_transform(df1["Sex"])
le_df = pd.DataFrame(data=res, columns=["Male"])

In [171]:
ohe = OneHotEncoder()
res1 = ohe.fit_transform(df1["Embarked"].to_numpy().reshape(-1,1))
ohe_df = pd.DataFrame(res1.toarray(), columns=ohe.get_feature_names_out(["Embarked"])).drop(columns="Embarked_C")

In [172]:
data = pd.concat([df1, le_df], axis=1).drop(columns='Sex')
data = pd.concat([data, ohe_df], axis=1).drop(columns="Embarked")

Теперь разбиваем датасет на таргет и признаки, разбиваем на тестовую и тренировочную выборки, фиксируем рандом стейт, и масштабируем выборки

In [173]:
y = data["Survived"]
X = data.drop(columns="Survived")

In [174]:
rs = 2025 #random state

In [175]:
X_train,  X_test, y_train, y_test = train_test_split(X,y, random_state=rs, test_size=0.15)

In [176]:
scaler = StandardScaler()
X_train_sc = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_sc = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# Обучение моделей, измерение качества

В качестве бейзлайн выберем модель, всегда предсказывающую класс 1. В качестве основной модели - Logistic Regression

In [177]:
dummy_clf = DummyClassifier(strategy="constant", constant=1, random_state=rs) # baseline
log_reg = LogisticRegression(random_state=rs)

In [178]:
dummy_clf.fit(X_train_sc, y_train)
log_reg.fit(X_train_sc, y_train)

y_pred_dummy = dummy_clf.predict(X_test_sc)
y_pred_log = log_reg.predict(X_test_sc)

In [179]:
print('F1-score for baseline: ', f1_score(y_test, y_pred_dummy))
print('ROC-AUC for baseline: ',  roc_auc_score(y_test, y_pred_dummy))


F1-score for baseline:  0.5540540540540541
ROC-AUC for baseline:  0.5


In [180]:
print('F1-score for model: ', f1_score(y_test, y_pred_log))
print('ROC-AUC for model: ',  roc_auc_score(y_test, y_pred_log))

F1-score for model:  0.7857142857142857
ROC-AUC for model:  0.8266814486326682


Видно, что логистическая регрессия предсказывает несколько лучше, чем бейзлайн.

Попробуем изменить порог бинаризации, чтобы улучшить результаты.

In [181]:
y_pred_log_new = (log_reg.predict_proba(X_test_sc)[:,1]>=0.55).astype(int)

In [182]:
print('Threshold = 0.55')
print('F1-score for model: ', f1_score(y_test, y_pred_log_new))
print('ROC-AUC for model: ',  roc_auc_score(y_test, y_pred_log_new))

Threshold = 0.55
F1-score for model:  0.8205128205128205
ROC-AUC for model:  0.8523651145602364


Действительно, результаты стали чуть лучше